In [9]:
#HouseKeeping
import numpy as np 
import pandas as pd 

data = pd.read_csv('TMNIST_Data.csv')

In [10]:
data = data.drop("names",axis=1)
data.head()

,labels,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#BASIC STUFF

data = np.array(data)
np.random.shuffle(data)
results = 10 #number of nodes output

test_data = data[0:1000].T
test_y  = test_data[0]
test_x  = test_data[1:]/255

train_data = data[1000:].T
train_y = train_data[0]
train_x = train_data[1:]/255

pixels, train_trials = train_x.shape

train_y.size

28900

In [5]:
def one_hot(Y):
    """
    Convert class labels to one-hot encoded vectors.

    Parameters
    ----------
    Y : numpy.ndarray
        Class labels (shape [batch_size,]).

    Returns
    -------
    numpy.ndarray
        One-hot encoded matrix (shape [num_classes, batch_size]).
    """
    Y_one_hot = np.zeros((Y.size, Y.max()+1)) #LEARN NEED (())
    Y_one_hot[np.arange(Y.size),Y] = 1    #LEARN
    return Y_one_hot.T

def derv_sigmoid(Z):
    return np.exp(-Z)/(1+np.exp(-Z))**2
def derv_ReLu(Z): 
    """
    Derivative of the ReLU function.

    Parameters
    ----------
    Z : numpy.ndarray
        Pre-activation input matrix.

    Returns
    -------
    numpy.ndarray
        Binary mask where 1 indicates Z > 0, else 0.
    """
    return Z > 0

def back_propogation(OUT, train_y, Z, A, params):
    """
    Compute gradients for all parameters using backpropagation.

    Parameters
    ----------
    OUT : numpy.ndarray
        Output probabilities from forward pass.
    train_y : numpy.ndarray
        Ground truth labels (shape [batch_size,]).
    Z : list of numpy.ndarray
        Pre-activations from forward pass.
    A : list of numpy.ndarray
        Activations from forward pass (including input).
    params : list of lists
        Current network parameters.

    Returns
    -------
    list of lists
        Gradients for each layer [dW, db], ordered from output to input.
    """
    Y = one_hot(train_y)
    m = train_y.size

    gradient = []

    dz = OUT - Y
    dw = 1/m * np.dot(dz, A[-2].T) 
    db = 1/m * np.sum(dz, axis=1, keepdims=True)
    gradient.append([dw, db])
    
    for i in range(1, len(params)):
        dw = 1/m * np.dot(dz, A[-i-1].T) 
        db = 1/m * np.sum(dz, axis=1, keepdims=True)
        gradient.append([dw, db])
    
    return gradient[::-1]  # Reverse to match params order

In [6]:
def get_accuracy(A,Y):
    """
    Calculate classification accuracy.

    Parameters
    ----------
    A : numpy.ndarray
        Predicted probabilities (shape [num_classes, batch_size]).
    Y : numpy.ndarray
        True labels (shape [batch_size,]).

    Returns
    -------
    float
        Accuracy between 0 and 1.
    """
    return sum(np.argmax(A,0) == Y)/Y.size #LEARN ARGMAX

def gradient_descent(H, N, X, Y, pixels, results, iterations, alpha):
    """
    Train the neural network using gradient descent.

    Parameters
    ----------
    H : int
        Number of hidden layers.
    N : int
        Neurons per hidden layer.
    X : numpy.ndarray
        Training data (shape [input_dim, batch_size]).
    Y : numpy.ndarray
        Training labels (shape [batch_size,]).
    pixels : int
        Input dimension.
    results : int
        Output dimension (number of classes).
    iterations : int
        Number of training iterations.
    alpha : float
        Learning rate.

    Returns
    -------
    list of lists
        Trained parameters for each layer.
    """
    params = init_parameters(H, N, pixels, results)
    for i in range(iterations):
        OUT, Z, A = forward_propogation(X, params)
        gradient = back_propogation(OUT, Y, Z, A, params)
        
        # Update parameters
        for j in range(len(params)):
            params[j][0] -= alpha * gradient[-j][0]
            params[j][1] -= alpha * gradient[-j][1]
        
        if i % 100 == 0:
            alpha = alpha
            print(f"Iteration:{i}")
            print(get_accuracy(A[-1], Y))

    return params

In [12]:
params = init_parameters(3,10,pixels,results)
OUT,Z,A = forward_propogation(train_x,params)
gradient = back_propogation(OUT,train_y,Z,A,params)

In [13]:
params = gradient_descent(3,10,train_x,train_y,pixels,results,1000,0.1)

Iteration:0
0.1002076124567474
Iteration:100
0.5698961937716263
Iteration:200
0.6311418685121107
Iteration:300
0.6578200692041523
Iteration:400
0.6812110726643599
Iteration:500
0.6997577854671281
Iteration:600
0.7177854671280277
Iteration:700
0.7283391003460208
Iteration:800
0.7352595155709343
Iteration:900
0.7463667820069204
